# 1000 Genomes analysis on BigQuery

### 0. Verily Workbench Introduction

This notebook, from the [VA Big Data Genomics Group](https://github.com/va-big-data-genomics), shows an example of using R for analysis of the 1000 Genomes BigQuery dataset. (It also illustrates how notebooks can be used to help reproduce publication results-- in this case for the 1000 Genomes phase 3 publication (https://doi.org/10.1038/nature15393)).

**This notebook is intended to be run on an R-based Verily Workbench workspace cloud environment**.
The notebook has been modified to automatically grab the Workbench workspace's underlying GCP project ID, but is otherwise unchanged.

**The original notebook source is here:  
https://github.com/va-big-data-genomics/variant-analysis-on-bigquery/blob/main/1000g_Analysis.ipynb**

Note that estimates of data processed are included below, which can be used to estimate costs.


### 1. Introduction

Authors: Prathima Vembu <br>
Notebook kernel: R (https://irkernel.github.io/installation/)<br>
Recommended Google Cloud Platform machine type: n1-standard-1 <br>
Predicted runtime: ~30 minutes <br>
Predicted cost for analysis <br>
1. Variant optimized table : 83.40 USD per one month (~100 USD including allele frequency computation) <br>
2. Sample optimized table : 0.00 USD per one month first 
*Note: 1 TB of query data processed per month is free* <br>

This notebook includes methods for reproducing several figures in the 1000 Genomes phase 3 publication (https://doi.org/10.1038/nature15393)<br>

##### 1.1: Comparing variant optimized table with sample optimized table
||Variant optimized|Sample optimized|
|---|---|---|
|User specified |No, generated by default|Yes, use ```--sample_lookup_optimized_output_table``` parameter|
|Number of tables created |25 (23 chromosome, sample_info, residual_info)|25 (23 chromosome, sample_info, residual_info)|
|Analysis |Variant lookup query - Find all variants within a enomic region|Sample lookup query - Find all variants in Sample 'Z'|
|Storage cost|No additional storage cost|Additional storage cost (for 25 tables), recovered when running queries for sample lookup analysis|
|Call field|Call field *not* unnested |Call field unnested|
|New field|No new column added|New column ```sample_id``` added which has the same value as ```call.sample_id``` column|
|1KG table size|1000_genomes_phase_3_variants_20150220 - 5.38 TB|1000_genomes_phase_3_optimized_schema_variants_20150220 - 1.94 TB|

##### 1.2: Table location 
Variant optimized table : ```bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220``` <br>
Sample optimized table : ```bigquery-public-data.human_genome_variants.1000_genomes_phase_3_optimized_schema_variants_20150220```


##### 1.3: Query cost comparison 

| Analysis | Variant table(1000Genomes variant table)| Data processed | Sample table(1000Genomes sample optimized table) | Data processed |
|---|---|---|---|---|
|Distinct sample_id’s|3.70 USD |1.74 TB | 0.46 USD|0.092 TB|
|Number of mapped and missing ID’s|3.70 USD|1.74 TB|0.46 USD|0.092 TB|
|Number of variant sites per sample| 19.00 USD|4.8 TB|1.27 USD|0.255 TB|
|Average singletons|19.00 USD|4.8 TB|1.28 USD|0.256 TB|
|Average singletons and rare variants |19.00 USD|4.8 TB|1.28 USD|0.256 TB|
|Distribution of Allele Frequency|0.00 USD|0.001 TB|0.00 USD|0.001 TB|
|Z-score computation|19.00 USD|4.8 TB|1.27 USD|0.255 TB|
|Total|83.4 USD|22.68 TB|6.02 USD|1.207 TB|


### 2. Installing necessary packages in R
Before starting analysis, install and load the below packages in R. 

In [ ]:
#helps connecting R to DBMS
install.packages("DBI") 
#helps to work with data on cloud storage
install.packages("bigrquery") 
#package to perform multiple data manipulation functions
install.packages("pillar") 
#package to restructure and aggregate dataframes
install.packages("reshape2") 
#package for creating graphics 
install.packages("ggplot2") 

library(DBI)
library(bigrquery)
library(pillar)
library(reshape2)
library(ggplot2)

### 3. Authenticate and connect to BigQuery 
Connect to the Google BigQuerydatabase account using ```dbconnect```. Replace ```BILLING_ACCOUNT``` with the name of the Google Cloud project you want charges to be billed to.

If you're running this notebook on a Verily Workbench workspace cloud environment, use the following cell to automatically grab the project ID. Otherwise, you will first need to set the `PROJECT_ID` manually.

In [ ]:
PROJECT_ID <- Sys.getenv('GOOGLE_CLOUD_PROJECT')
PROJECT_ID

In [ ]:
#dbconnect creates a connection to the DBMS
gcp_account<-dbConnect(bigquery(),"bigquery-public-data",
                       dataset="human_genome_variants", 
                       billing = PROJECT_ID)

### 4. Analysis

##### 4.1: Check to confirm the number of samples present in the BigQuery table (from the paper, n=2504). <br>
Two tables are available for the 1000 Genomes project which can be found under *bigquery-public-data.human_genome_variants* <br>
1. 1000_genomes_phase_3_variants_20150220 : The Variant table<br>
2. 1000_genomes_sample_info : The Sample table


In [ ]:
#Number of samples in the Variant table
#Runtime - 20s
#Data processed - 0.092 TB
variant_sample_count_sql<-"SELECT
COUNT(distinct call.name) AS Sample_count
FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v, UNNEST(v.call) AS call"

variant_sample_count<-dbGetQuery(gcp_account,variant_sample_count_sql)
variant_sample_count

In [ ]:
#Number of samples in the Sample table 
#Runtime - 1.8s
#Data processed - 30.76KB
sample_table_count_sql<-"SELECT
COUNT(Sample) AS Sample_count
FROM `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`"

sample_table_count<-dbGetQuery(gcp_account, sample_table_count_sql)
sample_table_count

Given that both these tables have different number of distinct sample ID's, it is important to make sure that the list of sample ID's in the ***variant table*** is a subset of the ID's from the ***sample table*** to prevent data loss. <br>
The query below computes the number of mapped ID's and number of missing ID's. All ID's from the ***variant table*** match with entries in the ***sample table***. 

In [ ]:
#Compute the number of mapped ID's and the number of missing ID's
#Runtime - 20s
#Data processed - 1.74TB
mapped_missing_values_sql<-"WITH cte1 AS (
SELECT DISTINCT call.name AS call_name
FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v,UNNEST(v.call) AS call
), cte2 AS (SELECT distinct Sample AS sample_name 
 FROM `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`
),cte3 AS(SELECT call_name, sample_name 
  FROM cte1 
  RIGHT JOIN cte2 
  ON cte1.call_name=cte2.sample_name
)
SELECT SUM(CASE WHEN call_name IS NULL THEN 1 ELSE 0 END) AS missing_values, 
SUM(CASE WHEN call_name IS NOT NULL THEN 1 ELSE 0 END) AS mapped_values FROM cte3"

mapped_missing_values<-dbGetQuery(gcp_account, mapped_missing_values_sql)
mapped_missing_values


##### 4.2: Number of variant sites per genome 
Refer: [Figure 1b](https://www.nature.com/articles/nature15393/figures/1)

In [ ]:
#Compute the number of variant sites per genome
#Runtime - 1.9s
#Data processed - 4.8 TB

variant_sites_per_genome_sql<-"SELECT s.Population,s.Sample,COUNT(reference_name) as variant_count,
FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v, UNNEST(v.call) AS call
INNER JOIN `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`s
ON call.name=s.Sample
WHERE EXISTS (SELECT gt FROM UNNEST(call.genotype) gt WHERE gt > 0 AND reference_name NOT IN ('X','Y','MT'))
GROUP BY 1,2 ORDER BY variant_count DESC"

variant_sites_per_genome<-dbGetQuery(gcp_account, variant_sites_per_genome_sql)
head(variant_sites_per_genome)

In [ ]:
#Runtime - 2.5s

#Make plots wider on jupyter notebooks
options(repr.plot.width=15, repr.plot.height=8)

#Set the order of the labels 
variant_sites_per_genome$Population<-factor(variant_sites_per_genome$Population,levels = unique(variant_sites_per_genome$Population))

#Generate the plot
ggplot(variant_sites_per_genome, aes(y=variant_count,x=Population, color=Population))+
geom_point()+theme(axis.text.x = element_blank(),axis.ticks.x = element_blank())+
scale_y_continuous(breaks = c(3800000,4000000,4200000,4400000,4600000,4800000,5000000,5200000,5400000,5600000),
                   minor_breaks = NULL, labels = function(l) {paste0(round(l/1e6,1),"M")})+
labs(title = "Variant sites per genome across populations",x = "Individual",y = "Variant sites")+
theme(plot.title = element_text(size=22),axis.text=element_text(size=14),axis.title=element_text(size=18))


##### 4.3: The average number of singletons per genome
Refer: [Figure 1c](https://www.nature.com/articles/nature15393/figures/1)


In [ ]:
#Compute the number of singletons per genome
#Runtime - 1.5s
#Data processed - 4.8TB
singletons_per_genome_sql<-"WITH cte1 AS (SELECT s.Population,Sample,
COUNT(reference_name) as variant_sites,COUNTIF(AC=1) AS Singletons,
FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v,
UNNEST(v.alternate_bases) AS Alternate, UNNEST(v.call) AS call
INNER JOIN `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`s
ON call.name=s.Sample
WHERE EXISTS(SELECT gt FROM UNNEST(call.genotype) gt WHERE gt>0 ) AND reference_name NOT IN ('X','Y','MT')
GROUP BY 1,2)
SELECT Population, AVG(Singletons) AS Average_Singletons FROM cte1 GROUP BY 1 ORDER BY 2 DESC"

singletons_per_genome<-dbGetQuery(gcp_account, singletons_per_genome_sql)
head(singletons_per_genome)


The output table is sorted in a descending order by the ***Average_Singletons*** column. Output dataframe has 2 columns, the ***Population*** group and the ***Average_Singletons*** computed.

In [ ]:
#Runtime - 0.9s

#Make plots wider on jupyter notebooks
options(repr.plot.width=15, repr.plot.height=8)

#Setting the labels, AC=1
singletons_per_genome$Population<-factor(singletons_per_genome$Population,levels = unique(singletons_per_genome$Population))

#Generate the plot, AC=1
ggplot(singletons_per_genome,aes(y=Average_Singletons,x=Population))+
geom_bar(aes(fill=Population),stat = "identity",position="dodge")+
scale_y_continuous(breaks = c(0,2000,4000,6000,8000,10000,12000,14000,16000,18000,20000,22000),
                   minor_breaks = NULL, labels = function(l) {paste0(round(l/1e3,1),"K")})+
labs(title = "Average number of singletons per genome",x = "Population",y = "Singletons per genome")+
theme(plot.title = element_text(size=22),axis.text=element_text(size=14),axis.title=element_text(size=18))



##### 4.4: Distribution of allele frequency 
Refer: Extended Figure 3a


Allele frequency has been extracted from the 1KG table and split into 20 buckets. This data is then plotted as a bar graph to see th allele frequency distribution. 

In [ ]:
#Bucketing the allele frequency
#Runtime - 16s
#Data processed - 650.58MB
af_sql<-"SELECT COUNT(AF) AF_Count, AF_bucket, 
FROM (SELECT  AF, CASE WHEN AF=0 THEN 0
                       WHEN AF>0 AND AF<=0.05 THEN 1
                       WHEN AF>0.05 AND AF<=0.1 THEN 2
                       WHEN AF>0.1 AND AF<=0.15 THEN 3
                       WHEN AF>0.15 AND AF<=0.2 THEN 4
                       WHEN AF>0.2 AND AF<=0.25 THEN 5
                       WHEN AF>0.25 AND AF<=0.3 THEN 6
                       WHEN AF>0.3 AND AF<=0.35 THEN 7
                       WHEN AF>0.35 AND AF<=0.4 THEN 8
                       WHEN AF>0.4 AND AF<=0.45 THEN 9
                       WHEN AF>0.45 AND AF<=0.5 THEN 10
                       WHEN AF>0.5 AND AF<=0.55 THEN 11
                       WHEN AF>0.55 AND AF<=0.6 THEN 12
                       WHEN AF>0.6 AND AF<=0.65 THEN 13
                       WHEN AF>0.65 AND AF<=0.7 THEN 14
                       WHEN AF>0.7 AND AF<=0.75 THEN 15
                       WHEN AF>0.75 AND AF<=0.8 THEN 16
                       WHEN AF>0.8 AND AF<=0.85 THEN 17
                       WHEN AF>0.85 AND AF<=0.9 THEN 18
                       WHEN AF>0.9 AND AF<=0.95 THEN 19
                       WHEN AF>0.95 AND AF<=1.0 THEN 20
                       ELSE -1 END AS AF_bucket
FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v,UNNEST(v.alternate_bases) AS Alternate )
GROUP BY AF_bucket
ORDER BY AF_bucket"

af<-dbGetQuery(gcp_account, af_sql)
af

In [ ]:
#Runtime - 0.4s

##REMOVING FIRST ROW - AF=0
#Make plots wider on jupyter notebooks
options(repr.plot.width=23, repr.plot.height=8)

#Removing AF=0 row 
af_mod = af[-1,]

#Generating the graph for variant tables 
ggplot(af_mod, aes(x=AF_bucket, y=AF_Count))+
geom_bar(stat = "identity",fill="red")+
labs(title = "Variant table - Allele Frequency distribution",x = "Allele frequency range",y = "Count(allele frequency)")+
theme(plot.title = element_text(size=22),axis.text=element_text(size=14),axis.title=element_text(size=18), 
      legend.title = element_blank(),legend.text = element_blank())+
scale_x_continuous(breaks=c(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20), minor_breaks=NULL,labels=c("0.05","0.1","0.15", "0.2", "0.25", "0.3", "0.35", "0.4", "0.45", 
      "0.5", "0.55", "0.6", "0.65", "0.7", "0.75", "0.8", "0.85", "0.9", "0.95","1.0"))+
geom_text(aes(label = scales::comma(AF_Count)),vjust=-0.2,size=5)


##### 4.5 : Number of rare variants and singletons per genome <br>
Refer: [Extended Figure 3b](https://www.nature.com/articles/nature15393/figures/7)


In [ ]:
#Compute the number of rare variants and singletons per genome
#Runtime - 43s
#Data processed - 4.8TB
rare_var_and_singletons_per_genome_sql<-"WITH cte1 AS (SELECT Population,Sample,
COUNT(reference_name) as variant_sites,
COUNTIF(AF<0.005) AS Rare_Variants,
COUNTIF(AC=1) AS Singletons,
FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v,
UNNEST(v.alternate_bases) AS Alternate, UNNEST(v.call) AS call
INNER JOIN `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`s
ON call.name=s.Sample
WHERE EXISTS(SELECT gt FROM UNNEST(call.genotype) gt WHERE gt>0 ) AND reference_name NOT IN ('X','Y','MT')
GROUP BY 1,2) SELECT Population, AVG(Rare_variants) AS Average_rare_variants, AVG(Singletons) AS Average_singletons 
FROM cte1 GROUP BY 1 ORDER BY Average_rare_variants,Average_singletons DESC"

rare_var_and_singletons_per_genome<-dbGetQuery(gcp_account, rare_var_and_singletons_per_genome_sql)
head(rare_var_and_singletons_per_genome)


The output table contains 3 colums, the ***Population***, ***Average_rare_variants*** computed and the ***Average_singletons*** computed. 

In [ ]:
#Runtime - 1.5s

#Make plots wider on jupyter notebooks
options(repr.plot.width=15, repr.plot.height=8)

#Set the order of the labels 
rare_var_and_singletons_per_genome$Population<-factor(rare_var_and_singletons_per_genome$Population,levels = unique(rare_var_and_singletons_per_genome$Population))

#Generating the graph 
ggplot(rare_var_and_singletons_per_genome,aes(x=Population, y=Average_rare_variants))+
geom_bar(stat = "identity", position = "stack", fill="grey")+
geom_bar(data = subset(rare_var_and_singletons_per_genome,variable=Average_singletons),
         aes(x=Population, y=Average_singletons, fill=Population),stat = "identity")+
scale_y_continuous(breaks = c(0,20000,40000,60000,80000,100000,120000,140000,160000,180000,200000,220000,
                              240000,260000,280000),labels = function(l) {paste0(round(l/1e3,1),"K")})+
labs(title = "Average number of rare variants (grey) with average number of singletons (coloured)",
     y = "Rare variants per genome")+
theme(plot.title = element_text(size=20),axis.text=element_text(size=14),axis.title=element_text(size=18))


##### 4.6: Computing the z score for variant count for each population
The z score will tell you how far a data point is from the mean. <br>
Z score was computed using the formula: 

Z = (X - μ) / 𝝈

Where:<br>
Z - standard score <br>
X - Observed value <br>
μ - Mean variant site count of all genomes (across populations included)<br>
𝝈 - Standard deviation computed across all genomes (all populations included)<br>

Refer: [Extended Figure 4](https://www.nature.com/articles/nature15393/figures/8)

In [ ]:
#Compute number of variant sites per genome 
#Runtime - 1m 23s
#Data processed - 4.78TB

z_score_sql<-"WITH cte1 AS (SELECT Population,Sample,
COUNT(reference_name) as variant_site,
FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v,UNNEST(v.alternate_bases) AS Alternate, 
UNNEST(v.call) AS call
INNER JOIN `bigquery-public-data.human_genome_variants.1000_genomes_sample_info`s
ON call.name=s.Sample
WHERE EXISTS(SELECT gt FROM UNNEST(call.genotype) gt WHERE gt>0 ) AND reference_name NOT IN ('X','Y','MT')
GROUP BY 1,2), cte2 AS (SELECT AVG(variant_site) AS total_mean, STDDEV(variant_site) AS total_sd FROM cte1), 
cte3 AS (SELECT Population, AVG(variant_site) AS sub_population_mean FROM cte1 GROUP BY 1) 
SELECT Population, (sub_population_mean-total_mean)/total_sd AS Z_score FROM cte2 join cte3 on 1=1 
GROUP BY 1,2 ORDER BY Z_score DESC"

z_score<-dbGetQuery(gcp_account, z_score_sql)
z_score

For the computation of the z score, the query has been broken into 4 sections:<br>
- Subquery 1: Join the variant and sample table based on sample id <br>
- Subquery 2: Compute the population mean and standard deviation across all samples (and sub-population groups)<br>
- Subquery 3: Compute the sample mean across variant sites per sub-population group <br>
- Finally, calulate the z score using the formula <br>

The output dataframe has 2 colums, the sub-population and the z-score calculated and will be used as the input dataframe to generate the below graph.  


In [ ]:
#Runtime - 0.7s

#Make plots wider on jupyter notebooks
options(repr.plot.width=17, repr.plot.height=8)

#Set the order of the labels 
z_score$Population<-factor(z_score$Population, levels = unique(z_score$Population))

#Generating the graph 
ggplot(z_score, aes(y=Z_score,x=Population))+
geom_bar(aes(fill=Population), stat = "identity", position = "dodge")+
theme(axis.text.x = element_blank(),axis.ticks.x = element_blank())+
scale_y_continuous(breaks = c(-2.0,-1.5,-1.0,-0.5,0,0.5,1.0,1.5,2.0),minor_breaks = NULL)+
labs(title = "Z score for variant count for each population",y = "Standardized number of sites/genome")+
theme(plot.title = element_text(size=20),axis.text=element_text(size=14),axis.title=element_text(size=18))


##### 4.7: Computation of allele frequency by changing syntax from Legacy SQL to Standard SQL 
Some important fucntions that have been used to compute allele frequency on the 1000 Genomes dataset
- __```LENGTH()``` function:__ 
    * This function computes the length of the array element provided
    * In this case we have used the ```alt``` as an input to the ```LENGTH()``` function
    * The function returns the length of the string in bytes. The equivalent of this in legacy SQL is ```POSITION()``` 
    * Usage: ```LENGTH(alt) AS alt``` 
    
    <br>
    <br>
    
- __```CAST()``` function :__
    * This function converts the value of any data type to the specified data type
    * We have used this function to convert the result obtained from ```LENGTH()``` into integer type
    * The equivalent of this in legacy SQL is ```INTEGER() WITHIN RECORD```  
    * Usage: ```SUM(CAST(genotype = alt AS INT64)) AS alt_count``` - This converts the values of ```alt``` from bytes to integer type

References: 
- [Million Veterean Program Codelabs](https://github.com/StanfordBioinformatics/mvp_aaa_codelabs/blob/master/sql)
- [Google genomics - BigQuery examples](https://github.com/deflaux/bigquery-examples/tree/master/1000genomes/sql)


In [ ]:
#Validating allele frequency computation code on 1000Genomes dataset
#Runtime - ~20 min 
#Data processed - 3.06 TB - costs $10.30  

allele_frequency_sql<-"WITH cte1 AS (SELECT reference_name, start_position, reference_bases, AF AS AF_thousand_genome , LENGTH(alt) AS alt,
         name,genotype, 
          FROM `bigquery-public-data.human_genome_variants.1000_genomes_phase_3_variants_20150220`v, 
          UNNEST(v.call) AS call, UNNEST(call.genotype) AS genotype, UNNEST(alternate_bases) AS alternate_bases
          ),cte2 AS (SELECT AF_thousand_genome, reference_name,reference_bases, alt,start_position,
                     SUM(CAST(genotype = 0 AS INT64)) AS ref_count, 
                     SUM( CAST(genotype = alt AS INT64)) AS alt_count,
                     FROM cte1 GROUP BY 1,2,3,4,5) SELECT reference_name, start_position, reference_bases,
                                          AF_thousand_genome,
                                          SUM(alt_count)/NULLIF(SUM(ref_count)+SUM(alt_count),0) AS alt_freq,
                                          SUM(ref_count)/NULLIF(SUM(ref_count)+SUM(alt_count),0) AS ref_freq,
                                          SUM(ref_count)+SUM(alt_count) AS num_sample_alleles,
                                          SUM(ref_count) AS ref_cnt,
                                          SUM(alt_count) AS alt_cnt,
                                          FROM cte2 GROUP BY 1,2,3,4"

allele_freqeuency<-dbGetQuery(gcp_account, allele_frequency_sql)
head(allele_freqeuency)